# 多层感知机简洁实现——逐行拆解

---

## 一、先说结论：这一节超级简单！

**这一节的全部内容就一句话：**

> 在 Softmax 分类的代码里，多加一个 Linear 层和一个 ReLU，就变成了多层感知机。

**和上一节的区别只有两行代码。** 其他全部一样。

---

## 二、完整代码 + 逐行解释

### 第一块：导入（和之前完全一样）

```python
import torch
from torch import nn
from d2l import torch as d2l
```

> 没有任何新东西，跳过。

---

### 第二块：搭建模型（唯一的重点！）

```python
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
```

#### 和上一节（Softmax分类）的对比

```python
# 上一节（Softmax 分类）：
net = nn.Sequential(
    nn.Flatten(),           # 拍平
    nn.Linear(784, 10)      # 直接 784 → 10
)

# 这一节（多层感知机）：
net = nn.Sequential(
    nn.Flatten(),           # 拍平（一样）
    nn.Linear(784, 256),    # 784 → 256  ← ⭐⭐新加的！
    nn.ReLU(),              # 激活函数    ← ⭐⭐新加的！
    nn.Linear(256, 10)      # 256 → 10（从256开始，不是784了）
)
```

**区别只有中间多了两行：**

```
多了 nn.Linear(784, 256)   ← 隐藏层
多了 nn.ReLU()             ← 激活函数
```

#### 逐层解释

**第 0 层：`nn.Flatten()`**

```
和之前一模一样
[256, 1, 28, 28] → [256, 784]
把图片拍成一条线
```

**第 1 层：`nn.Linear(784, 256)`** ← 新的！

```
这就是"隐藏层"

输入：784 个数字（拍平后的图片像素）
输出：256 个数字（隐藏层的 256 个神经元）

内部自动创建：
  权重 w1：形状 [256, 784] → 256×784 = 200704 个参数
  偏置 b1：形状 [256]      → 256 个参数

计算：H = X × w1ᵀ + b1
```

**⚠️ 为什么是 256？**

```
256 是你自己选的！
可以换成 128、512 都行
这就是"超参数"

经验：一般选 2 的幂次（64, 128, 256, 512）
     因为 GPU 对这些数字计算更快
```

**第 2 层：`nn.ReLU()`** ← 新的！

```
激活函数！打破线性！

ReLU(x) = max(0, x)
  正数 → 保持原值
  负数 → 变成 0

例子：
  输入 [0.5, -1.2, 3.0, -0.3, 2.1, ...]  （256个数字）
  输出 [0.5,  0.0, 3.0,  0.0, 2.1, ...]  （负数全变0）

形状不变：[256, 256] → [256, 256]
只是把里面的负数变成了 0
```

**如果没有这一层会怎样？（上一节讲过）**

```
没有 ReLU：
  两层 Linear 合并成一层 → 等于白加
  还是线性模型，不能画曲线

有 ReLU：
  真正的两层网络！能画曲线了！
```

**第 3 层：`nn.Linear(256, 10)`**

```
输出层

输入：256 个数字（隐藏层的输出）
输出：10 个数字（10 个类别的分数）

注意⭐⭐⭐上一节是 nn.Linear(784, 10)
      这一节是 nn.Linear(256, 10)
                         ↑
                  从隐藏层出来的是 256，不是 784 了
```

#### 完整数据流

```
图片 [256, 1, 28, 28]        ← 一批 256 张图
         │
         ▼ Flatten
    [256, 784]                ← 每张拍成 784 个数
         │
         ▼ Linear(784, 256)
    [256, 256]                ← 变成 256 个数（隐藏层）
         │
         ▼ ReLU
    [256, 256]                ← 负数变 0（形状不变）
         │
         ▼ Linear(256, 10)
    [256, 10]                 ← 变成 10 个分数（每类一个）
         │
         ▼
    10 个分数 → 哪个最大就是哪一类
```

---

### 第三块：初始化权重（照抄就行，不用记）

```python
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);
```

```
和上一节一模一样
对每一个 Linear 层的权重用正态分布初始化
不用记，照抄
```

---

### 第四块：训练（和之前完全一样！）

```python
batch_size, lr, num_epochs = 256, 0.1, 10
```

```
三个变量写在一行（Python 语法，等价于）：
  batch_size = 256      ← 每次取 256 张图
  lr = 0.1              ← 学习率
  num_epochs = 10       ← 训练 10 轮
```

```python
loss = nn.CrossEntropyLoss(reduction='none')
```

```
分类问题 → 交叉熵损失
和上一节一模一样
```

```python
trainer = torch.optim.SGD(net.parameters(), lr=lr)
```

```
SGD 优化器
net.parameters() 现在包含更多参数了：
  上一节只有：w[784×10] + b[10] = 7850 个参数
  这一节有：  w1[784×256] + b1[256] + w2[256×10] + b2[10]
            = 200704 + 256 + 2560 + 10
            = 203530 个参数！

参数多了很多，但用法完全一样
trainer 会帮你管理所有参数
```

```python
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
```

```
加载数据（和之前一模一样）
```

```python
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)
```

```
调用书里的训练函数
内部做的事情你已经知道了：

for epoch in range(10):
    for X, y in train_iter:
        l = loss(net(X), y)    # 前向传播 + 算损失
        trainer.zero_grad()     # 清梯度
        l.backward()            # 反向传播
        trainer.step()          # 更新参数
```

---

## 三、这一节和之前每一节的对比

```python
# ======= 线性回归 =======
net = nn.Sequential(
    nn.Linear(2, 1)
)
loss = nn.MSELoss()

# ======= Softmax 分类 =======
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 10)          # ← 只有这一层
)
loss = nn.CrossEntropyLoss()

# ======= 多层感知机（这一节）=======
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 256),        # ← 新加的隐藏层
    nn.ReLU(),                  # ← 新加的激活函数
    nn.Linear(256, 10)
)
loss = nn.CrossEntropyLoss()    # ← 和 Softmax 一样！
```

**看到了吗？每一节只多了一点点东西，其他全是复用！**

```
线性回归          →  Softmax 分类      →  多层感知机
                   （加了 Flatten）      （加了隐藏层 + ReLU）
                   （换了损失函数）       （模型更深了，其他不变）
```

---

## 四、你需要掌握的 vs 不需要掌握的

| 🔴 必须掌握 | 🟢 不用管 |
|------------|----------|
| 模型结构多了 `Linear + ReLU` | `init_weights` 怎么写 |
| 隐藏层大小自己选（超参数） | `reduction='none'` |
| ReLU 放在两个 Linear 之间 | `d2l.train_ch3` 内部实现 |
| 数据流的形状变化 | 画图代码 |
| 其他部分和 Softmax 完全一样 | |

---

## 五、你要背的模板

```python
# ========== 多层感知机万能模板 ==========

net = nn.Sequential(
    nn.Flatten(),                       # 🔒 图片拍平
    nn.Linear(通道×高×宽, 隐藏层大小),     # ✏️ 隐藏层
    nn.ReLU(),                          # 🔒 激活函数
    nn.Linear(隐藏层大小, 类别数)          # ✏️ 输出层
)

loss = nn.CrossEntropyLoss()            # 🔒 分类用这个
trainer = torch.optim.SGD(net.parameters(), lr=✏️学习率)

# 训练循环（完全一样）
for epoch in range(✏️轮数):
    for X, y in train_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
```

**和 Softmax 模板的区别只有 `net` 里面多了两行。其他一字不差。**

---

## 六、自查题

### 题目：手写多层感知机

FashionMNIST，隐藏层 128 个神经元，10 类分类。

```python
net = nn.Sequential(
    _______________,
    _______________,
    _______________,
    _______________
)
```

<details>
<summary>答案</summary>

```python
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)
```
</details>

**答对了 → 这一节完全过关！真的就这么简单！**